In [8]:
from datetime import datetime
from sdv.metadata import SingleTableMetadata
# load measurement as a pandas dataframe
import pandas as pd
import numpy as np

# load the data
df = pd.read_csv('108522_last_4_6.csv', delimiter=';')

df['Time'] = df['Time'].apply(
            lambda x: datetime.utcfromtimestamp(x / 1000).strftime('%Y-%m-%d %H:%M:%S'))

# manually add a sequence key to the data (108522)
df['sequence_key'] = 108522

# write json specs
import json

# Load metadata
with open('metadata_PAR.json', 'r') as f:
    metadata_dict = json.load(f)
    
metadata = SingleTableMetadata.load_from_dict(metadata_dict)

In [9]:
from sdv.sequential import PARSynthesizer

synthesizer = PARSynthesizer(metadata)
synthesizer.fit(df)

synthetic_data = synthesizer.sample(num_sequences=1)

C:\Users\FelixNeubauer\anaconda3\envs\sdv\lib\site-packages\deepecho\models\par.py:132: RuntimeWarning: Mean of empty slice
  'mu': np.nanmean(x[i]),
C:\Users\FelixNeubauer\anaconda3\envs\sdv\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [10]:
synthetic_data.head()

,sequence_key,Time,Interval,Min op-reachability (%),Max op-reachability (%),Avg op-reachability (%),Min in-octets (kbit/s),Max in-octets (kbit/s),Avg in-octets (kbit/s),Min in-utilization (%),...,Avg out-octets (kbit/s),Min out-utilization (%),Max out-utilization (%),Avg out-utilization (%),Min out-errors (%),Max out-errors (%),Avg out-errors (%),Min out-discards (%),Max out-discards (%),Avg out-discards (%)
0,256169871,2024-02-28 22:50:00,2432752,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,2.742524,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
1,256169871,2024-02-28 22:55:00,16415083,100.0,100.0,100.0,NaN,24.335330,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
2,256169871,2024-02-29 01:51:42,12172644,100.0,100.0,NaN,0.0,13.136324,4.109874,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,256169871,2024-02-29 07:41:41,29700000,NaN,100.0,NaN,NaN,NaN,NaN,NaN,...,2.840227,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
4,256169871,2024-02-29 08:51:39,300000,NaN,NaN,100.0,NaN,23.652863,2.775929,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN


In [12]:
from sdmetrics.reports.single_table import QualityReport

report = QualityReport()
report.generate(df, synthetic_data, metadata_dict)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 30/30 [00:00<00:00, 1038.67it/s]|
Column Shapes Score: 90.67%



C:\Users\FelixNeubauer\anaconda3\envs\sdv\lib\site-packages\sdmetrics\single_column\statistical\kscomplement.py:59: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  statistic, _ = ks_2samp(real_data, synthetic_data)


(2/2) Evaluating Column Pair Trends: |██████████| 435/435 [00:01<00:00, 369.44it/s]|
Column Pair Trends Score: 85.22%

Overall Score (Average): 87.94%



In [13]:
fig = report.get_visualization(property_name='Column Shapes')
fig.show()

In [14]:
# save df and synthetic data to csv
synthetic_data.to_csv('108522_last_4_6_synthetic.csv', index=False)